In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Prepare libraries
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import sklearn
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS 
import plotly.express as px

# **Reading in data**

In [ ]:
base_url = '/content/drive/My Drive/550_project/'
data_21_1 = pd.read_csv(base_url + "2021-1.csv")
data_20_1 = pd.read_csv(base_url + "2020-1.csv")
data_19_1 = pd.read_csv(base_url + "2019-1.csv")
data_18_1 = pd.read_csv(base_url + "2018-1.csv")
# data_21_2 = pd.read_csv(base_url + "2021-2.csv")
# data_20_2 = pd.read_csv(base_url + "2020-2.csv")
# data_19_2 = pd.read_csv(base_url + "2019-2.csv")
# data_18_2 = pd.read_csv(base_url + "2018-2.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (2,3,4,14,19,20,23,25,29,31,33,34,35,36,40,45,54,55,56,61,65,67,77,78,83,85,87,89,91) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (45,55,56,83,85,87,89,91) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (45,54,55,56,83,85,87,89,91) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (3,4,26,32,33,34,35,36,40,43,44,45,54,55,56,65,67,71,74,76,77,78,80,82,83,85,87,89,91) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
print(data_19_1.columns)

Index(['assistance_transaction_unique_key', 'assistance_award_unique_key',
       'award_id_fain', 'modification_number', 'award_id_uri', 'sai_number',
       'federal_action_obligation', 'total_obligated_amount',
       'non_federal_funding_amount', 'total_non_federal_funding_amount',
       'face_value_of_loan', 'original_loan_subsidy_cost',
       'total_face_value_of_loan', 'total_loan_subsidy_cost',
       'disaster_emergency_fund_codes_for_overall_award',
       'outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw',
       'obligated_amount_funded_by_COVID-19_supplementals_for_overall_a',
       'action_date', 'action_date_fiscal_year',
       'period_of_performance_start_date',
       'period_of_performance_current_end_date', 'awarding_agency_code',
       'awarding_agency_name', 'awarding_sub_agency_code',
       'awarding_sub_agency_name', 'awarding_office_code',
       'awarding_office_name', 'funding_agency_code', 'funding_agency_name',
       'funding_sub_agency

In [ ]:
# Only keep the 38 desired columns in our schema
cols_to_keep = ['assistance_transaction_unique_key', 'award_id_fain', 'federal_action_obligation', 
                'total_obligated_amount', 'non_federal_funding_amount', 'total_non_federal_funding_amount', 'disaster_emergency_fund_codes_for_overall_award',
                'obligated_amount_funded_by_COVID-19_supplementals_for_overall_a', 'action_date', 'action_date_fiscal_year', 'period_of_performance_start_date',
                'period_of_performance_current_end_date', 'awarding_agency_code', 'awarding_agency_name', 'funding_agency_code', 
                'funding_agency_name', 'object_classes_funding_this_award', 'program_activities_funding_this_award', 
                'recipient_duns', 'recipient_name', 'recipient_parent_duns', 'recipient_parent_name', 'recipient_country_code', 
                'recipient_country_name', 'recipient_city_code', 'recipient_city_name', 'recipient_state_code', 'recipient_state_name',
                'recipient_zip_code', 'primary_place_of_performance_scope', 'cfda_number', 'cfda_title', 'assistance_type_code', 
                'assistance_type_description', 'award_description', 'action_type_code', 'action_type_description']
df_21_1 = data_21_1[cols_to_keep]
df_20_1 = data_20_1[cols_to_keep]
df_19_1 = data_19_1[cols_to_keep]
df_18_1 = data_18_1[cols_to_keep]
# df_21_2 = data_21_2[cols_to_keep]
# df_20_2 = data_20_2[cols_to_keep]
# df_19_2 = data_19_2[cols_to_keep]
# df_18_2 = data_18_2[cols_to_keep]

In [ ]:
df_21_1.shape

(1000000, 37)

In [ ]:
# Join the four tables into one big table
list_of_data = [df_21_1, df_20_1, df_19_1, df_18_1]
df_all = pd.concat(list_of_data, axis=0)

In [ ]:
df_all.shape # Test that it successfully joined without dropping any columns

(4000000, 37)

In [ ]:
# Check that no primary keys are null
pd.isna(df_all['assistance_transaction_unique_key']).unique()

array([False])

In [ ]:
pd.isna(df_all['award_id_fain']).unique()

array([False,  True])

In [ ]:
pd.isna(df_all['obligated_amount_funded_by_COVID-19_supplementals_for_overall_a']).unique()
df_test1 = df_all[pd.notnull(df_all['obligated_amount_funded_by_COVID-19_supplementals_for_overall_a'])]

In [ ]:
df_test1.head()

# **Fixing problematic columns**

In [ ]:
# It makes sense to manually fill in the missing values for all "obligated fund amounts" column because if it's missing it should be 0
# Obligated for covid19
df_all['obligated_amount_for_covid19'] = df_all['obligated_amount_funded_by_COVID-19_supplementals_for_overall_a'].apply(lambda x: 0 if pd.isnull(x) else x)
df_all = df_all.drop(columns=['obligated_amount_funded_by_COVID-19_supplementals_for_overall_a'])

In [ ]:
# pd.isna(df_all['federal_action_obligation']).unique() 
# Manually fill in 0 for Nan values of federal_action_obligation
x = df_all['federal_action_obligation'].apply(lambda x: 0 if pd.isnull(x) else x)
df_all['federal_action_obligation'] = x

In [ ]:
# pd.isna(df_all['total_obligated_amount']).unique()
# Manually fill in 0 for Nan values of total_obligated_amount
x = df_all['total_obligated_amount'].apply(lambda x: 0 if pd.isnull(x) else x)
df_all['total_obligated_amount'] = x

In [ ]:
# pd.isna(df_all['non_federal_funding_amount']).unique()
# Manually fill in 0 for Nan values of non_federal_funding_amount
x = df_all['non_federal_funding_amount'].apply(lambda x: 0 if pd.isnull(x) else x)
df_all['non_federal_funding_amount'] = x

In [ ]:
# pd.isna(df_all['total_non_federal_funding_amount']).unique()
# Manually fill in 0 for Nan values of total_non_federal_funding_amount
x = df_all['total_non_federal_funding_amount'].apply(lambda x: 0 if pd.isnull(x) else x)
df_all['total_non_federal_funding_amount'] = x

In [ ]:
# Manually fill disaster code (String "none" if it's Nan)
x = df_all['disaster_emergency_fund_codes_for_overall_award'].apply(lambda x: "None" if pd.isnull(x) else x)
df_all['disaster_emergency_fund_codes_for_overall_award'] = x

In [ ]:
# Recipient_parent_duns fill in the dun/name of themselves if it's nan (they don't have a parent)
# x = df_all[['recipient_parent_duns', 'recipient_duns']].apply(lambda r: r['recipient_duns'] if pd.isnull(r['recipient_parent_duns']) else r['recipient_parent_duns'], axis=1)
x = df_all['recipient_parent_duns'].apply(lambda x: "None" if pd.isnull(x) else x)
df_all['recipient_parent_duns'] = x

In [ ]:
# Recipient_parent_name, do same as above
# x = df_all[['recipient_parent_name', 'recipient_name']].apply(lambda r: r['recipient_name'] if pd.isnull(r['recipient_parent_name']) else r['recipient_parent_name'], axis=1)
x = df_all['recipient_parent_name'].apply(lambda x: "None" if pd.isnull(x) else x)
df_all['recipient_parent_name'] = x

In [ ]:
# It should be ok to have NULL for some cols, but not others
### Things that can't be null:
'''
'assistance_transaction_unique_key', 'award_id_fain', 'action_date', 'action_date_fiscal_year', 'period_of_performance_start_date',
                    'period_of_performance_current_end_date', 'awarding_agency_code', 'awarding_agency_name', 'funding_agency_code',
                    'funding_agency_name', 'object_classes_funding_this_award', 'recipient_duns', 'recipient_name', 
                    'recipient_country_code', 'recipient_country_name', 'recipient_city_code', 'recipient_city_name', 
                    'recipient_state_code', 'recipient_state_name', 'recipient_zip_code', 'primary_place_of_performance_scope', 
                    'cfda_number', 'assistance_type_code', 'assistance_type_description', 'award_description', 
                    'action_type_code', 'action_type_description'
'''
list_of_not_null = ['assistance_transaction_unique_key', 'award_id_fain', 'federal_action_obligation', 
                'total_obligated_amount', 'non_federal_funding_amount', 'total_non_federal_funding_amount', 'disaster_emergency_fund_codes_for_overall_award',
                'obligated_amount_for_covid19', 'action_date', 'action_date_fiscal_year', 'period_of_performance_start_date',
                'period_of_performance_current_end_date', 'awarding_agency_code', 'awarding_agency_name', 'funding_agency_code', 
                'funding_agency_name', 'object_classes_funding_this_award', 'program_activities_funding_this_award', 
                'recipient_duns', 'recipient_name', 'recipient_parent_duns', 'recipient_parent_name', 'recipient_country_code', 
                'recipient_country_name', 'recipient_city_code', 'recipient_city_name', 'recipient_state_code', 'recipient_state_name',
                'recipient_zip_code', 'primary_place_of_performance_scope', 'cfda_number', 'cfda_title', 'assistance_type_code', 
                'assistance_type_description', 'award_description', 'action_type_code', 'action_type_description']
df_all = df_all.dropna()

In [ ]:
pd.isna(df_all['recipient_parent_duns']).unique()

array([False])

In [ ]:
df_all.shape

(119987, 37)

In [ ]:
df_all.to_csv(base_url + "df_all.csv", index=False)

In [ ]:
# df1 = df.loc[df['Date'] > 'Feb 06, 2019']
df_all2.head()

In [ ]:
# Use group by to split them up into multiple tables
countries = df_all2[['recipient_country_code', 'recipient_country_name']]
countries.head()

,recipient_country_code,recipient_country_name
0,USA,UNITED STATES
1,USA,UNITED STATES
2,USA,UNITED STATES
3,USA,UNITED STATES
4,USA,UNITED STATES


In [ ]:
countries.groupby(by=['recipient_country_code', 'recipient_country_name']).count().reset_index()

,recipient_country_code,recipient_country_name
0,USA,UNITED STATES


In [ ]:
states = df_all2[['recipient_state_code', 'recipient_state_name']]
states_final = states.groupby(by=['recipient_state_code', 'recipient_state_name']).count().reset_index()

In [ ]:
len(states_final['recipient_state_code'].unique())
len(states_final['recipient_state_name'].unique())
# They are both 56, so match

In [ ]:
cities = df_all2[['recipient_city_code', 'recipient_city_name']]
cities.head()

,recipient_city_code,recipient_city_name
0,50000.0,WASHINGTON
1,77364.0,SUSANVILLE
2,36780.0,LA CONNER
3,58550.0,OUZINKIE
4,1990.0,ANACORTES


In [ ]:
cities.groupby(by=['recipient_city_code']).max().reset_index() # One city code matches multiple cities, so I arbitrarily picked the alphabetical "max" city if there are multiple matches


,recipient_city_code,recipient_city_name
0,00100,ACHILLE
1,00124,ABBEVILLE
2,00125,ABILENE
3,00135,ABINGTON
4,00145,ABINGTON
...,...,...
10207,9875.0,BURLINGTON
10208,988.0,ALBERTVILLE
10209,9880.0,BURNS
10210,9928.0,BRADY
